In [ ]:
#get all the submissions from the day 
#get 10 with most twitter citations
#summarize and post them 

In [2]:
"""
based on ET - every day one hour past 00:00 past ET i will tweet it 

go to certain website 

1. get date of day before 
2. open first page and check if date is equals to target date if yes add all the paper urls from the page to the list and if date is older then target date exit
3. open all pages, get the number of tweet impressions, get the url to the pdf, title and subject area
4. make summary and post
"""
from datetime import datetime, timedelta
import pytz
import requests
from bs4 import BeautifulSoup
import time

# Get date from yesterday ET time
eastern = pytz.timezone('US/Eastern')
yesterday = datetime.now(eastern) - timedelta(days=1)
yesterday_date = yesterday.strftime('%Y-%m-%d')
print("Yesterday's date in ET:", yesterday_date)

page_data = []

def iterate_biorxiv_pages():
    page_number = 0
    while True:
        url = f"https://www.biorxiv.org/content/early/recent?page={page_number}"
        print(url)
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Get the date based on the X path
        date_element = soup.select_one('#block-system-main > div > div > div > div:nth-child(2) > div:nth-child(1) > div > div > div > div > div:nth-child(1) > h3')
        date_text = date_element.get_text(strip=True)
        
        # Reformat the date to match the format of our yesterday_date
        page_date = datetime.strptime(date_text, '%B %d, %Y').strftime('%Y-%m-%d')

        print("page date: ", page_date)
        
        # Check if the date is older than the target date
        if page_date < yesterday_date:
            break
        
        # Skip the page if the date is newer than the target date
        if page_date > yesterday_date:
            page_number += 1
            continue

        # Extract all the URLs from the page
        article_blocks = soup.find_all('div', class_='highwire-cite highwire-cite-highwire-article highwire-citation-biorxiv-article-pap-list-overline clearfix')
        
        for block in article_blocks:
            title_element = block.find('span', class_='highwire-cite-title')
            if title_element:
                link_element = title_element.find('a', class_='highwire-cite-linked-title')
                if link_element and 'href' in link_element.attrs:
                    paper_url = "https://www.biorxiv.org" + link_element['href']
                    page_data.append(paper_url)
        
        #here i can extract all the data

        # If date is target date, get all the data and save it in list
        print(page_date)
        # Add logic to extract and save data here
        
        page_number += 1
    return page_data

page_data = iterate_biorxiv_pages()

Yesterday's date in ET: 2024-08-30
https://www.biorxiv.org/content/early/recent?page=0
page date:  2024-08-30
2024-08-30
https://www.biorxiv.org/content/early/recent?page=1
page date:  2024-08-30
2024-08-30
https://www.biorxiv.org/content/early/recent?page=2
page date:  2024-08-30
2024-08-30
https://www.biorxiv.org/content/early/recent?page=3
page date:  2024-08-30
2024-08-30
https://www.biorxiv.org/content/early/recent?page=4
page date:  2024-08-30
2024-08-30
https://www.biorxiv.org/content/early/recent?page=5
page date:  2024-08-30
2024-08-30
https://www.biorxiv.org/content/early/recent?page=6
page date:  2024-08-30
2024-08-30
https://www.biorxiv.org/content/early/recent?page=7
page date:  2024-08-30
2024-08-30
https://www.biorxiv.org/content/early/recent?page=8
page date:  2024-08-30
2024-08-30
https://www.biorxiv.org/content/early/recent?page=9
page date:  2024-08-30
2024-08-30
https://www.biorxiv.org/content/early/recent?page=10
page date:  2024-08-30
2024-08-30
https://www.biorxi

In [7]:
import asyncio
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeoutError
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import nest_asyncio

# Apply the nest_asyncio patch
nest_asyncio.apply()

# Initialize an empty list to store the dictionaries
tweet_data_list = []

# Function to generate the target date
def get_yesterday_date():
    yesterday = datetime.now() - timedelta(days=1)
    return yesterday.strftime('%Y-%m-%d')

yesterday_date = get_yesterday_date()

# Asynchronous function to fetch and parse a single URL
async def fetch_and_parse(url, context):
    print(f"Fetching URL: {url}")
    retries = 3
    for attempt in range(retries):
        try:
            page = await context.new_page()
            await page.goto(url, timeout=60000)  # Wait up to 60 seconds for the page to load
            await page.wait_for_selector('#count_twitter', timeout=60000)  # Wait up to 60 seconds for the element
            content = await page.content()
            await page.close()
            break  # Exit the retry loop if successful
        except PlaywrightTimeoutError as e:
            print(f"Timeout error on {url}: {e}")
        except Exception as e:
            print(f"Error on {url}: {e}")
        if attempt < retries - 1:
            print(f"Retrying {url} (attempt {attempt + 1}/{retries})")
        else:
            print(f"Failed to fetch {url} after {retries} attempts")
            return

    soup = BeautifulSoup(content, 'html.parser')
    
    # Extract the date from the page
    date_element = soup.select_one('#block-system-main > div > div > div > div > div:nth-child(2) > div > div > div:nth-child(3) > div')
    if date_element:
        date_text = date_element.get_text(strip=True)
        # Remove the 'Posted\xa0' prefix if it exists
        if date_text.startswith('Posted\xa0'):
            date_text = date_text.replace('Posted\xa0', '')
        # Remove the period at the end of the date string
        if date_text.endswith('.'):
            date_text = date_text[:-1]
        try:
            page_date = datetime.strptime(date_text, '%B %d, %Y').strftime('%Y-%m-%d')
            print(f"Parsed page date: {page_date}")  # Debug print
        except ValueError as e:
            print(f"Error parsing date on {url}: {e}")
            return
        
        # Check if the date matches the target date
        if page_date == yesterday_date:
            print(f"Date matches target date: {page_date}")  # Debug print
            # Extract the number of tweets using a CSS selector
            tweet_element = soup.select_one('#count_twitter')
            print("tweet_element: ", tweet_element)  # Debug print
            if tweet_element:
                tweet_count = tweet_element.get_text(strip=True)
                print(f"Extracted tweet count: {tweet_count}")  # Debug print
                
                # Create a dictionary with the URL as key and tweet count as value
                tweet_data = {url: tweet_count}
                
                # Add the dictionary to the list
                tweet_data_list.append(tweet_data)
            else:
                print(f"Tweet element not found on {url}")
        else:
            print(f"Date does not match target date on {url}")
    else:
        print(f"Date element not found on {url}")

# Main asynchronous function to handle multiple requests in batches
async def main(urls, batch_size=10, delay=5):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        
        for i in range(0, len(urls), batch_size):
            batch = urls[i:i + batch_size]
            print(f"Processing batch: {batch}")
            context = await browser.new_context()
            tasks = [fetch_and_parse(url, context) for url in batch]
            await asyncio.gather(*tasks)
            await context.close()
            await asyncio.sleep(delay)  # Add a delay between batches
        
        await browser.close()

# List of URLs to process
urls = page_data

# Run the main function
await main(urls)

# Print the list of dictionaries
print(tweet_data_list)

Processing batch: ['https://www.biorxiv.org/content/10.1101/2024.08.30.610472v1', 'https://www.biorxiv.org/content/10.1101/2024.08.30.610471v1', 'https://www.biorxiv.org/content/10.1101/2024.08.30.609158v1', 'https://www.biorxiv.org/content/10.1101/2024.08.30.609923v1', 'https://www.biorxiv.org/content/10.1101/2024.08.30.610437v1', 'https://www.biorxiv.org/content/10.1101/2024.08.30.610386v1', 'https://www.biorxiv.org/content/10.1101/2024.08.30.610159v1', 'https://www.biorxiv.org/content/10.1101/2024.08.30.610496v1', 'https://www.biorxiv.org/content/10.1101/2024.08.30.610464v1', 'https://www.biorxiv.org/content/10.1101/2024.08.30.609148v1']
Fetching URL: https://www.biorxiv.org/content/10.1101/2024.08.30.610472v1
Fetching URL: https://www.biorxiv.org/content/10.1101/2024.08.30.610471v1
Fetching URL: https://www.biorxiv.org/content/10.1101/2024.08.30.609158v1
Fetching URL: https://www.biorxiv.org/content/10.1101/2024.08.30.609923v1
Fetching URL: https://www.biorxiv.org/content/10.1101/2

In [12]:
# Function to find the top ten URLs with the most tweets
def get_top_ten_tweets(tweet_data_list):
    top_ten_tweets = []
    
    # Flatten the list of dictionaries into a single list of tuples (url, tweet_count)
    flattened_tweets = [(url, int(tweet_count)) for data in tweet_data_list for url, tweet_count in data.items()]
    
    # Sort the list in descending order based on tweet count
    sorted_tweets = sorted(flattened_tweets, key=lambda x: x[1], reverse=True)
    
    # Select the top ten URLs with the most tweets
    top_ten_tweets = sorted_tweets[:10]
    
    return top_ten_tweets

# Get the top ten URLs with the most tweets
top_ten_tweets = get_top_ten_tweets(tweet_data_list)
print("Top ten URLs with the most tweets:")
for url, tweet_count in top_ten_tweets:
    print(f"URL: {url}, Tweets: {tweet_count}")

Top ten URLs with the most tweets:
URL: https://www.biorxiv.org/content/10.1101/2023.11.27.568910v2, Tweets: 103
URL: https://www.biorxiv.org/content/10.1101/2022.01.22.477348v2, Tweets: 65
URL: https://www.biorxiv.org/content/10.1101/2024.08.28.610193v1, Tweets: 44
URL: https://www.biorxiv.org/content/10.1101/2024.05.08.593115v2, Tweets: 37
URL: https://www.biorxiv.org/content/10.1101/2024.02.26.582181v2, Tweets: 33
URL: https://www.biorxiv.org/content/10.1101/2024.06.10.598294v4, Tweets: 28
URL: https://www.biorxiv.org/content/10.1101/2024.08.29.610422v1, Tweets: 22
URL: https://www.biorxiv.org/content/10.1101/2024.08.30.610287v1, Tweets: 20
URL: https://www.biorxiv.org/content/10.1101/2024.08.28.610199v1, Tweets: 17
URL: https://www.biorxiv.org/content/10.1101/2022.07.13.499031v2, Tweets: 16


In [ ]:
#now we can get